In [ ]:
import numpy as np

In [ ]:
arr = np.ones(93)
arr

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# Load audio (mono for simplicity)
filename = "../Songs/rufus_du_sol--innerbloom.mp3"  # replace with path
y, sr = librosa.load(filename, sr=None)

# Tempo & beat tracking
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
beat_times = librosa.frames_to_time(beat_frames, sr=sr)

print(f"Estimated tempo: {tempo:.2f} BPM")
print("Beat times (s):", beat_times[:20], "...")

# Visualization
plt.figure(figsize=(14, 6))
librosa.display.waveshow(y, sr=sr, alpha=0.6)
plt.vlines(beat_times, -1, 1, color='r', alpha=0.8, linestyle='--', label='Beats')
plt.title(f"Waveform with Detected Beats (Tempo: {tempo:.1f} BPM)")
plt.legend()
plt.show()


In [ ]:
tempo

In [ ]:
beat_frames